# glyphs

> Contains the Glyph class used to define the different type of glyphs that can be used to represent features, as well the basic plotting functions for GenomeBrowser

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp glyphs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from bokeh.io import output_notebook #|hide_line
output_notebook(hide_banner=True) #|hide_line

In [ ]:
#| export
import numpy as np
import pandas as pd
import io

from bokeh.plotting import figure
from bokeh.models.tools import BoxZoomTool
from bokeh.models import HoverTool, NumeralTickFormatter, LabelSet
from bokeh.models.glyphs import Patches
from bokeh.models import (
    CustomJS,
    Range1d,
    ColumnDataSource,
)
from genomenotebook.data import get_example_data_dir
from genomenotebook.utils import (
    parse_gff,
    default_types,
    default_attributes,
)

from collections import defaultdict
import os
from typing import *
import copy

In [ ]:
#| export
from collections import defaultdict

In [ ]:
#| export
#| hide
Y_RANGE = (0, 1)
def get_y_range() -> tuple:
    """Accessor that returns the Y range for the genome browser plot
    """
    return Y_RANGE

In [ ]:
#| export
#| hide
def arrow_coordinates(feature, 
                      height: float = 1, #relative height of the feature (between 0 and 1)
                      feature_height: float = 0.15, #fraction of the annotation track occupied by the feature glyphs
                      ):
    
    feature_size = feature.right - feature.left
    
    if feature.strand=="+":
        arrow_base = feature.end - np.minimum(feature_size, 100)
    else:
        arrow_base = feature.end + np.minimum(feature_size, 100)
    
    xs=(feature.start,
        feature.start,
        arrow_base,
        feature.end,
        arrow_base
       )
    
    offset=feature_height*(1-height)/2
    y_min = 0.05+offset
    y_max = 0.05+feature_height-offset
    ys = (y_min, y_max, y_max, (y_max + y_min) / 2, y_min)
    return xs, ys


In [ ]:
#| export
#| hide
def box_coordinates(feature, 
                    height: float = 1, #relative height of the feature (between 0 and 1)
                    feature_height: float = 0.15, #fraction of the annotation track occupied by the feature glyphs
                    ):
    xs=(feature.left, feature.left,
        feature.right, feature.right)
    
    offset=feature_height*(1-height)/2
    y_min = 0.05+offset
    y_max = 0.05+feature_height-offset
    ys = (y_min, y_max, y_max, y_min)
    return xs, ys

In [ ]:
#| export
class Glyph:
    def __init__(self,
                 glyph_type: str ="arrow", # type of the Glyph (arrow or box)
                 colors: tuple = ("purple","orange"), # can be a single color or a tuple of two colors, one for each strand
                 alpha: float = 0.8, #transparency
                 show_name: bool = True, #
                 height: float = 1,  #height of the feature relative to other features (between 0 and 1)
                 ):
        """A class used to define the different types of glyphs shown for different feature types."""
        self.glyph_type=glyph_type
        if type(colors)==str:
            self.colors=(colors,)
        else:
            self.colors=colors

        assert alpha>=0 and alpha <=1
        self.alpha=alpha
        self.show_name=show_name
        assert height>0 and height<=1
        self.height=height

        if glyph_type == "box":
            self.coordinates = box_coordinates
        else:
            self.coordinates = arrow_coordinates

    def get_patch(self,
                  feature, # row of a pandas DataFrame extracted from a GFF file
                  feature_height: float = 0.15, #fraction of the annotation track height occupied by the features
                  ):
    
        if len(self.colors)>1:
            color_dic={"+":self.colors[0],
                    "-":self.colors[1]}
        else:
            color_dic=defaultdict(lambda: self.colors[0])

        return self.coordinates(feature, self.height, feature_height), color_dic[feature.strand], self.alpha
    
    def copy(self):
        return copy.deepcopy(self)
    
    def __repr__(self) -> str:
        attributes = ["glyph_type","colors","height","alpha","show_name"]
        r=f"Glyph object with attributes:\n"
        for attr in attributes:
            r+=f"\t{attr}: {getattr(self, attr)}\n"
        return r

In [ ]:
#| export
def get_default_glyphs():
    """"Returns a dictionnary with:
            - keys: feature types
            - values: a Glyph object
    """
    basic_arrow=Glyph(glyph_type="arrow",colors=("purple","orange"),alpha=0.8,show_name=True)
    basic_box=Glyph(glyph_type="box",colors=("grey",),alpha=1,height=0.8,show_name=False)
    
    default_glyphs=defaultdict(lambda: basic_arrow.copy()) #the default glyph will be the same as for CDS etc.
    default_glyphs.update(dict([(f,basic_arrow.copy()) for f in ["CDS", "ncRNA", "rRNA", "tRNA"]]))
    default_glyphs['repeat_region']=basic_box.copy()
    default_glyphs['exon']=basic_box.copy()
    return default_glyphs

default_glyphs=get_default_glyphs()

In [ ]:
default_glyphs['CDS']

Glyph object with attributes:
	glyph_type: arrow
	colors: ('purple', 'orange')
	height: 1
	alpha: 0.8
	show_name: True

In [ ]:
#| export
def get_patch_coordinates(feature, glyphs_dict, feature_height=0.15):
    glyph=glyphs_dict[feature.type]
    return glyph.get_patch(feature, feature_height=feature_height)

In [ ]:
data_path = get_example_data_dir()
gff_path = os.path.join(data_path, "MG1655_U00096.gff3")
features=parse_gff(gff_path, 
                   feature_types=default_types, 
                   seq_id="U00096.3")

features.head().apply(get_patch_coordinates,glyphs_dict=default_glyphs, axis=1)

0    (((190, 190, 190, 255, 190), (0.05, 0.2, 0.2, ...
1    (((337, 337, 2699, 2799, 2699), (0.05, 0.2, 0....
2    (((2801, 2801, 3633, 3733, 3633), (0.05, 0.2, ...
3    (((3734, 3734, 4920, 5020, 4920), (0.05, 0.2, ...
4    (((5234, 5234, 5430, 5530, 5430), (0.05, 0.2, ...
dtype: object

In [ ]:
#| export
def get_feature_name(feature,
                     glyphs_dict,
                     name="gene",
                     attributes: list = default_attributes,
                    ) -> str:
    """Gets the name of the feature to be displayed. If the Glyph for the feature type has the attribute show_name=False then an empty string is returned.
    If name is not an attribute of the feature, then the first attribute in the attributes list is used.
    """
    if glyphs_dict[feature.type].show_name:
        if hasattr(feature,name):
            if feature[name]:
                return feature[name]
        
        for attr in attributes:
            if feature[attr]:
                return feature[attr]
        
        return feature[9]
    else:
        return ""

In [ ]:
features.head().apply(get_feature_name,name="gene",glyphs_dict=default_glyphs, axis=1)

0    thrL
1    thrA
2    thrB
3    thrC
4    yaaX
dtype: object

In [ ]:
#| export
def get_feature_patches(features: pd.DataFrame, #DataFrame of the features 
                        left: int, #left limit
                        right: int, #right limit
                        glyphs_dict: dict, #a dictionnary of glyphs to use for each feature type
                        attributes: list = default_attributes, #list of attributes to display when hovering
                        name: str = default_attributes[0], #attribute to be displayed as the feature name
                        feature_height: float = 0.15, #fraction of the annotation track height occupied by the features
                       )->pd.DataFrame:
    features=features.loc[(features["right"] > left) & (features["left"] < right)]
    
    if len(features)>0:
        coordinates, colors, alphas = zip(*features.apply(get_patch_coordinates,glyphs_dict=glyphs_dict,feature_height=feature_height,axis=1))
        xs, ys = zip(*coordinates)
    else:
        colors = []
        xs, ys = [], []
        
    names=list(features.apply(get_feature_name,
                         name=name,
                         glyphs_dict=glyphs_dict,
                         axis=1))
        
    out=dict(names=list(names),
             xs=list(xs),
             ys=list(ys),
             color=list(colors),
             alpha=list(alphas),
             pos=list(features.middle.values),
            )
    for attr in attributes:
        if attr in features.columns:
            values=features[attr].fillna("").astype(str)
            out[attr]=values.to_list() #tried to split long strings here but Bokeh then ignores it 
            
    return pd.DataFrame(out)

In [ ]:
get_feature_patches(features,8000,12000, glyphs_dict=default_glyphs, name="gene")

,names,xs,ys,color,alpha,pos,gene,locus_tag,product
0,talB,"(8238, 8238, 9091, 9191, 9091)","(0.05, 0.2, 0.2, 0.125, 0.05)",purple,0.8,8714.5,talB,b0008,transaldolase B
1,mog,"(9306, 9306, 9793, 9893, 9793)","(0.05, 0.2, 0.2, 0.125, 0.05)",purple,0.8,9599.5,mog,b0009,molybdopterin adenylyltransferase
2,satP,"(10494, 10494, 10028, 9928, 10028)","(0.05, 0.2, 0.2, 0.125, 0.05)",orange,0.8,10211.0,satP,b0010,acetate/succinate:H(+) symporter
3,yaaW,"(11356, 11356, 10743, 10643, 10743)","(0.05, 0.2, 0.2, 0.125, 0.05)",orange,0.8,10999.5,yaaW,b0011,putative enzyme-specific chaperone YaaW
4,mbiA,"(10830, 10830, 11215, 11315, 11215)","(0.05, 0.2, 0.2, 0.125, 0.05)",purple,0.8,11072.5,mbiA,b0012,uncharacterized protein MbiA
5,yaaI,"(11786, 11786, 11482, 11382, 11482)","(0.05, 0.2, 0.2, 0.125, 0.05)",orange,0.8,11584.0,yaaI,b0013,DUF2541 domain-containing protein YaaI


In [ ]:
#| export
def create_genome_browser_plot(glyphSource, 
                               x_range, 
                               attributes=default_attributes, 
                               height = 150, 
                               label_angle = 45,
                               label_font_size = "10pt",
                               output_backend = "webgl",
                               feature_height = 0.15,
                               **kwargs):
    

    y_min, y_max = get_y_range()
    p_annot = figure(
        tools = "xwheel_zoom, xpan, save, reset",
        active_scroll = "xwheel_zoom",
        height = height,
        x_range = x_range,
        y_range = Range1d(y_min, y_max),
        output_backend=output_backend,
        **kwargs
    )
    # Add tool
    p_annot.add_tools(BoxZoomTool(dimensions="width"))

    # Format x axis values
    p_annot.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
    # Hide grid
    p_annot.xgrid.visible = False
    p_annot.ygrid.visible = False
    # Hide axis
    p_annot.yaxis.visible = False
    glyph = p_annot.add_glyph(
        glyphSource, Patches(xs="xs", ys="ys", fill_color="color", fill_alpha="alpha")
    )
    # gene labels in the annotation track
    # This seems to be necessary to show the labels
    p_annot.scatter(x="pos", y=0, size=0, source=glyphSource)
    labels = LabelSet(
        x="pos",
        y=feature_height+0.07,
        text="names",
        level="glyph",
        angle=label_angle,
        text_font_size=label_font_size,
        x_offset=-5,
        y_offset=0,
        source=glyphSource,
        text_align='left',
    )

    p_annot.add_layout(labels)
    tooltips=[(attr,f"@{attr}") for attr in attributes]
    p_annot.add_tools(
        HoverTool(
            renderers=[glyph],
            tooltips=tooltips,
        )
    )
    return p_annot

In [ ]:
from bokeh.models import (
    Range1d,
    ColumnDataSource,
)
from bokeh.plotting import show

In [ ]:
features = parse_gff(gff_path,
                     feature_types=default_types,
                    )
x_range = Range1d(
            20000, 40000, 
            bounds=(0,50000), 
            max_interval=100000,
            min_interval=40
        )

feature_patches = get_feature_patches(features, x_range.start, x_range.end, glyphs_dict=default_glyphs)
glyph_source = ColumnDataSource(feature_patches)

p = create_genome_browser_plot(glyph_source, x_range, height=200)
show(p)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()